In [76]:
import os
import getpass

import pandas as pd
import matplotlib.pyplot as plt

import requests
import json

import asyncio

from typing import Sequence

import autogen
from autogen_core.models import UserMessage
from autogen_core.tools import FunctionTool
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination, TimeoutTermination, MaxMessageTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage, TextMessage, MultiModalMessage
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_agentchat.base import TaskResult
from autogen_core import CancellationToken

from transformers import pipeline

In [77]:
# Load your OPENAI API key
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

# Load your Spoonacular API key
if not os.environ.get('SPOONACULAR_API_KEY'):
    os.environ['SPOONACULAR_API_KEY'] = getpass.getpass("Enter your Spoonacular API key: ")

In [78]:
# Create OpenAI model client for inference
openai_model_client = OpenAIChatCompletionClient(
    model = "gpt-4o-mini",
    api_key = os.environ.get("OPENAI_API_KEY")
)

SPOONACULAR_API_BASE_URL = f'https://api.spoonacular.com/recipes/complexSearch'

param_schema = pd.read_csv('SpoonacularAPI_InputGuide.csv').to_dict(orient = 'records')

In [79]:
# Define all model tools
async def search_web_tool(params: str) -> str:
    """Searches web for relevant information."""
    with open('log.txt', 'a') as f:
        f.write(f'Web search params: {params} | {type(params)}\n')
    api_params = json.loads(params)
    api_params['apiKey'] = os.environ.get('SPOONACULAR_API_KEY')
    # api_params['addRecipeInformation'] = True
    results = requests.get(SPOONACULAR_API_BASE_URL, params = api_params)
    recipes = results.json().get('results', [])

    return json.dumps(recipes)

def log_report(report: str):
    """Logs the report to a file."""
    with open('log_report.txt', 'a') as f:
        f.write(f'Report: {report}\n')
        f.write('-' * 50 + '\n')

In [ ]:
user_agent = UserProxyAgent(
    name = "UserProxyAgent",
    description = "Proxy agent for user interactions.",
)

parser_agent_system_message = f"""
    You are a strict parameter extraction agent. 
    You will be given a natural language query and a list of valid parameters, including name, type, example, and description. 
    Your job is to return a JSON dictionary where each key is the exact 'Name' of a parameter and each value is type-safe and based on the user's query. 
    Do not include keys that are not present in the schema. 
    Only use parameters relevant to the query. 
    If the query is vague, only use 'query'.
    
    The parameter guidelines are:
    {param_schema}

    The 'number' parameter is REQUIRED to be greater than 0 and less than 6. If
    the user's request is unclear, default to 3. Always include the 'number' parameter.
    The user may use text numbers (e.g. 'two' or 'six') instead of numeric values. Be sure to convert them to numeric values.
    The 'query' parameter is REQUIRED and should be a string. Always include the 'query' parameter.

    You should take particular note to fill the `cuisine`, `excludeCuisine`, `diet`, `includeIngredients`, and `excludeIngredients` parameters.
    Each of the aforementioned should be filled in if they have any relevance to the user query. 

    If the usery query is "Find me paleo Thai chicken recipes without peanuts", you should return:
    {{
        "query": "Thai chicken recipes without peanuts",
        "number": 3,
        "cuisine": "thai",
        "includeIngredients": "chicken",
        "diet": "paleo",
        "excludeIngredients": "peanuts"
    }}

    If the user query is "six recipes of pasta and beef without peas", you should return:
    {{
        "query": "pasta and beef",
        "number": 6,
        "includeIngredients": "pasta,beef",
        "excludeIngredients": "peas"
    }}

    If the user query is "vegan dishes with pepper", you should return:
    {{
        "query": "vegan dishes with pepper",
        "number": 3,
        "diet": "vegan",
        "includeIngredients": "pepper"
    }}

    If the user query is "quick eats with at least 30 carbs and at most 80 protein", you should return:
    {{
        "query": "quick eats",
        "number": 3,
        "minCarbs": 30,
        "maxProtein": 80
    }}

    All other parameters are OPTIONAL and should be included only if they are relevant to the user's query.

    You should return a dictionary in the following format:
    {{"parameter_name": "parameter_value", ...}}

    The parameter_name should be the exact name of the parameter in the schema.
    The parameter_value should be the value that is relevant to the user's query.

    Do not include any other text or explanation.
    If you cannot find a parameter that matches the query, return an empty dictionary.
    If the query is not valid, return an empty dictionary.
    If the query is too vague, return an empty dictionary.
    """

parser_agent = AssistantAgent(
    name = 'ParserAgent',
    model_client = openai_model_client,
    description = 'An agent designed to turn NLP queries into API calls. It will take a user query and a parameter schema, and return a list of parameters that can be used to call the API. The agent will use the parameter schema to determine which parameters are required for the API call. The agent will also use the user query to determine which parameters are relevant for the API call. The agent will return a list of parameters that can be used to call the API.',
    system_message = parser_agent_system_message
)

web_search_agent = AssistantAgent(
    name = "WebSearchAgent",
    model_client = openai_model_client,
    description = "A web search agent for relevant results.",
    tools = [search_web_tool],
    system_message = """
        You are a web search agent.
        You have one tool: `search_web_tool`, which takes a **single JSON string** as input.
        Your job is to take the **entire output of the previous agent**, without modifying it, and pass it directly as the parameter to the tool.

        Do not interpret, summarize, rephrase, or change the contents of the previous message.
        Assume the previous message is a valid JSON dictionary string, and use it as-is.

        Example:
        If the previous agent says:
        {"query": "cheesy burger", "minCalories": 300, "number": 2}
        Then you must call:
        search_web_tool({"params": "{\"query\": \"chicken and pasta\", \"number\": 3}"})
        You only make one search call at a time.
        Once you have results, you will never do calculations based on them.
    """
)

prioritizer_agent = AssistantAgent(
    name="PrioritizerAgent",
    model_client = openai_model_client,
    description="A prioritizer agent. Useful for analyzing semantics of items relating to a natural language query.",
    system_message=f"""
        You are an expert semantic evaluator agent.
        Given the list of items that you have received, you should analyze the data and provide a ranked list of items that are most relevant to the user's query.
        All items should be unedited and in the same format as they were received.
        The only difference in your output is that the items should be in a ranked order, with the most relevant item first.
        You will generate a relevance score between 0% - 100% for each item and use that as the basis for your ranking.
        You should also provide a short explanation of why you ranked the items in that order.
    """,
)

writer_agent = AssistantAgent(
    name = "WriterAgent",
    model_client = openai_model_client,
    description = "A writer agent. All writing should be in English, make sense, and be perfectly clear.",
    tools = [log_report],
    system_message = """
        You are an expert writer, on par with the best in the world.
        Given the data that you have received, you should write a full, cohesive, and ranked report about it that is directly in line with the user's request.
        You should emphasize and display adherence to all user filters that have been requested.
        This writing should be visually appearing, contain vivid imagery, and be engaging.

        You must log this entire report by passing your entire report to the 'log_report' function. 
        Do not make any alterations to the report - pass it in its entirety to log_report.
    """
)

In [ ]:
# Set up the multi-agent team, rotation, and termination conditions

text_mention_termination = TextMentionTermination("APPROVE")

team = RoundRobinGroupChat(
    participants = [user_agent, parser_agent, web_search_agent, prioritizer_agent, writer_agent], 
    termination_condition = text_mention_termination
)

In [82]:
with open('log_report.txt', 'w') as f:
    f.write("")

# Run a task through the multi-agent system
await team.reset()

task = ""

await Console(team.run_stream(task = task))

---------- user ----------

---------- UserProxyAgent ----------
chicken and pasta
---------- ParserAgent ----------
{
    "query": "chicken and pasta",
    "number": 3,
    "includeIngredients": "chicken,pasta"
}
---------- WebSearchAgent ----------
[FunctionCall(id='call_OPwpmCrafDOrTgIhvxwz79yl', arguments='{"params":"{\\"query\\":\\"chicken and pasta\\",\\"number\\":3,\\"includeIngredients\\":\\"chicken,pasta\\"}"}', name='search_web_tool')]
---------- WebSearchAgent ----------
[FunctionExecutionResult(content='[{"id": 632778, "title": "Artisan Farfalle Pasta With Smoked Salmon and Cream Sauce", "image": "https://img.spoonacular.com/recipes/632778-312x231.jpg", "imageType": "jpg"}, {"id": 606953, "title": "Cajun Chicken Pasta", "image": "https://img.spoonacular.com/recipes/606953-312x231.jpg", "imageType": "jpg"}, {"id": 655575, "title": "Penne Pasta with Broccoli and Cheese", "image": "https://img.spoonacular.com/recipes/655575-312x231.jpg", "imageType": "jpg"}]', name='search_web

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='', type='TextMessage'), UserInputRequestedEvent(source='UserProxyAgent', models_usage=None, metadata={}, request_id='d5557b45-8f37-483e-92d0-2e607380bc6c', content='', type='UserInputRequestedEvent'), TextMessage(source='UserProxyAgent', models_usage=None, metadata={}, content='chicken and pasta', type='TextMessage'), TextMessage(source='ParserAgent', models_usage=RequestUsage(prompt_tokens=4744, completion_tokens=31), metadata={}, content='{\n    "query": "chicken and pasta",\n    "number": 3,\n    "includeIngredients": "chicken,pasta"\n}', type='TextMessage'), ToolCallRequestEvent(source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=288, completion_tokens=36), metadata={}, content=[FunctionCall(id='call_OPwpmCrafDOrTgIhvxwz79yl', arguments='{"params":"{\\"query\\":\\"chicken and pasta\\",\\"number\\":3,\\"includeIngredients\\":\\"chicken,pasta\\"}"}', name='search_web_tool')], type='T